In [0]:
def load_measurements(filename, fmode):
  
  import numpy as np
  tvec = np.array([])
  data = np.array([])

  for line in open(filename):
    line = line.rstrip()
    line = line.split(',')
    line = [float(value) for value in line]
    tvec = np.append(tvec, line[:6])
    data = np.append(data, line[6:])

  tvec = tvec.reshape(int(np.size(tvec) / 6), 6)
  data = data.reshape(int(np.size(data) / 4), 4)

  if fmode == 'forward fill':
    for row in range(len(data)):
      for column in range(len(data[row])):
        if data[row, column] == -1:

          if row != 0:
            data[row, column] = data[row - 1, column]
          
          if row == 0:
            print('Error! You can not forward fill the first row.')
  
  if fmode == 'backward fill':
    for row in range(len(data)):
      for column in range(len(data[row])):
        if data[row, column] == -1:

          if row != len(data) - 1:
            for next_row in range(row, len(data)):
              if data[next_row, column] != -1:
                data[row, column] = data[next_row, column]
                continue
          
          if row == len(data) - 1:
            print('Error! You can not backward fill the last row.')
  
  if fmode == 'drop':
    for row in data:
      for column in row:
        if column == -1:
          
          data = np.delete(data, row, 0)

  return (tvec, data)

In [0]:
def aggregate_measurements(tvec, data, period):

  import numpy as np
  tvec_a = np.array([])
  data_a = np.array([])

  if period == 'month':
    for row in range(len(tvec)):
      
      aggregation = np.array([])
      for periods in range(len(tvec)):
        if tvec[row][0] == tvec[periods][0] and tvec[row][1] == tvec[periods][1]:
          tvec_a = np.append(tvec_a, tvec[row])
          aggregation = np.append(aggregation, data[row])

      aggregation = aggregation.reshape(int(np.size(aggregation) / 4), 4)
      sums = []
      sums.append(sum(aggregation[:, 0]))
      sums.append(sum(aggregation[:, 1]))
      sums.append(sum(aggregation[:, 2]))
      sums.append(sum(aggregation[:, 3]))
      data_a = np.append(data_a, sums)

  data_a = data_a.reshape(int(np.size(data_a) / 4), 4)

  return (tvec_a, data_a)

In [0]:
def aggregate_measurements(tvec, data, period):

  import numpy as np
  tvec_a = np.array([])
  data_a = np.array([])

  if period == 'month':
    for row in range(len(tvec)):
      
      aggregation = np.array([])
      for periods in range(len(tvec)):
        if tvec[row][0] == tvec[periods][0] and tvec[row][1] == tvec[periods][1]:
          tvec_a = np.append(tvec_a, tvec[row])
          aggregation = np.append(aggregation, data[row])

      aggregation = aggregation.reshape(int(np.size(aggregation) / 4), 4)
      sums = []
      sums.append(sum(aggregation[:, 0]))
      sums.append(sum(aggregation[:, 1]))
      sums.append(sum(aggregation[:, 2]))
      sums.append(sum(aggregation[:, 3]))
      data_a = np.append(data_a, sums)

  data_a = data_a.reshape(int(np.size(data_a) / 4), 4)

  return (tvec_a, data_a)

In [0]:
def print_statistics(tvec, data):

  headers = ['Zone', 'Minimum', '1. quart', '2. quart', '3. quart', 'Maximum']
  table = []

  import numpy as np
  for index in range(4):
    row = []
    row.append(index + 1)
    row.append(min(data[:, index]))
    row.append(np.quantile(data[:, index], 0.25))
    row.append(np.median(data[:, index]))
    row.append(np.quantile(data[:, index], 0.75))
    row.append(max(data[:, index]))
    table.append(row)

  row = []
  row.append('All')
  row.append(min([index[1] for index in table]))
  row.append(np.quantile([index[2] for index in table], 0.25))
  row.append(np.median([index[3] for index in table]))
  row.append(np.quantile([index[4] for index in table], 0.75))
  row.append(max([index[5] for index in table]))
  table.append(row)

  from tabulate import tabulate
  print(tabulate(table, headers))

Main script

In [0]:
print('Welcome to our data analysis program!')
loaded = False
period = False

while True:

  options = ['load data', 'aggregate data', 'display statistics',
  'visualize electricity consumption', 'documentation', 'quit']

  print('Please input one of the following options:' + '\n')
  for option in options:
    print(option.title())

  option = input('\n' + 'Input: ').lower()

  if option not in options:
    print('\n' + 'Error! Your input "' + option + '" is not an option.' + '\n')

  if option == 'load data':
    
    print('\n' + 'Please input the full path to the file:' + '\n')
    filename = input('Input: ')

    options = ['fill forward', 'fill backward', 'delete corrupt measurements']
    print('\n' + 'Please input one of the following options for handling corrupted measurements:' + '\n')
    for option in options:
      print(option.title())
    
    fmode = input('\n' + 'Input: ').lower()

    if fmode not in options:
      print('\n' + 'Error! Your input "' + fmode + '" is not an option.' + '\n')
      continue

    try:
      print('\n' + 'Loading the file ...' + '\n')
      tvec, data = load_measurements(filename, fmode)
    except:
      print('Error! You have entered an invalid file path.' + '\n')
      continue
    
    loaded = True
    print('You have succesfully loaded the file!' + '\n')
  
  if option == 'aggregate date':

    if loaded != True:
      print('\n' + 'Error! You can not display statistics before loading a file.' + '\n')
      continue
    
    options = ['consumption per minute (no aggregation)', 'consumption per hour',
    'consumption per day', 'consumption per month', 'hour-of-day consumption (hourly average)']

    for option in options:
      print(option.title())

    period = input('\n' + 'Input: ').lower()

    if period not in options:
      print('\n' + 'Error! Your input "' + period + '" is not an option.' + '\n')
      continue
    
    tvec, data = aggregate_measurements(tvec, data, period)

  if option == 'display statistics':
    
    if loaded != True:
      print('\n' + 'Error! You can not display statistics before loading a file.' + '\n')
      continue
    
    if period != False:
      print(period.title())
    
    print('\n')
    print_statistics(tvec, data)
    print('\n')
  
  if option == 'visualize electricity consumption':

    if loaded != True:
      print('\n' + 'Error! You can not display statistics before loading a file.' + '\n')
      continue
    
    options = ['each zone', 'all zones']
    print('\n' + 'Please input one of the following options for visualizing consumption:' + '\n')
    for option in options:
      print(option.title())
    
    zone = input('\n' + 'Input: ').lower()

    if zone not in options:
      print('\n' + 'Error! Your input "' + zone + '" is not an option.' + '\n')
      continue
    
    if period == False:
      print('\n' + 'Error! You can not visualize consumption before aggregating data.' + '\n')
      continue
    
    import matplotlib.pyplot as plt
    
    if zone == 'each zone':
      b = 1
    
    if zone == 'all zones':
      a = 1

  if option == 'documentation':
    
    from tabulate import tabulate
    print('\n' + tabulate([
    ['Load Data', 'Load the data from a file into the program.'],
    ['Aggregate Data', 'Aggregate the data based on your period of choice.'],
    ['Display Statistics', 'Display a table of statistics about your data.'],
    ['Visualize Electricity Consumption', 'Plot your consumption of electricity.'],
    ['Documentation', 'Display this table.'],
    ['Quit', 'Terminate the program.']
    ], ['Option', 'Description']
    ) + '\n')

  if option == 'quit':

    print('\n' + 'Thank you for making us your choice of data analysis program!')
    print('Program has been succesfully terminated.')
    break

Welcome to our data analysis program!
Please input one of the following options:

Load Data
Aggregate Data
Display Statistics
Visualize Electricity Consumption
Documentation
Quit

Input: load data

Please input the full path to the file:

Input: /content/drive/My Drive/Colab Notebooks/DTU/2008.csv

Please input one of the following options for handling corrupted measurements:

Fill Forward
Fill Backward
Delete Corrupt Measurements

Input: fill forward

Loading the file ...

You have succesfully loaded the file!

Please input one of the following options:

Load Data
Aggregate Data
Display Statistics
Visualize Electricity Consumption
Documentation
Quit

Input: display statistics


Zone      Minimum    1. quart    2. quart    3. quart    Maximum
------  ---------  ----------  ----------  ----------  ---------
1              -1           0         0         0           80
2              -1           0         0         1           76
3              -1           0         1        17       

KeyboardInterrupt: ignored